<a href="https://colab.research.google.com/github/priya-200/data-Science-projects/blob/master/Stock%20Price%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stock Price Prediction using LSTM

This project is to enhance my skill in the field of Deep learning and Data Science.
We are goin to predict the stock Price of APPLE company

## Import statements

In [1]:
import tensorflow as tf
import keras
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## Data Collection
Here we are going to fetch the data of APPLE stock price using the API provided by yahoo via python librar

In [40]:
#Fetch the data from yfinance API
aapl_data = yf.download('AAPL',start = '2020-01-01',end = '2024-01-01')

# Display few rows
aapl_data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.059425,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.349136,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.925621,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.582672,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.750237,132079200


## Preprocessing our data
This is a crucial step in a machine learning or a deep learning process because a improper data may affect our model to a great extend

In [41]:
# Check for missing value

aapl_data.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

## Normalization
It is nessasary because our model likes to see the data in the range 0 and 1, which may increase it's performance

In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
aapl_data_scaled = scaler.fit_transform(aapl_data['Close'].values.reshape(-1,1))

In the above step we reshaped the data because initially the data was in the form of 1D array but fit_transform need the data in the form 2D array

In [ ]:
scaler.fit_transform(aapl_data['Close'].values)

ValueError: Expected 2D array, got 1D array instead:
array=[ 75.08750153  74.35749817  74.94999695 ... 193.1499939  193.58000183
 192.52999878].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
aapl_data['Close'].values.shape

(1006,)

In [ ]:
aapl_data['Close'].values.reshape(-1,1).shape

(1006, 1)

## Creating Sequence
Since we plan to use RNN networks along with LSTM we need the input data to be in sequential Order
Sequence Length:
This is really a arbitary number where we say the model to look n last data.
For example if the number is 60 then the model will look for last 60 days data for evaluate a data

In [43]:
X = []
y = []
for i in range(60,len(aapl_data_scaled)):
  X.append(aapl_data_scaled[i-60:i,0])
  y.append(aapl_data_scaled[i,0])

In [44]:
# You might be wondering what is it mean by aapl_data_scaled[i-60:i,0]
# Let's see it in action

aapl_data_scaled[0:60].shape
# The shape is in 2D array noe to conver it into 1D array we use 0

(60, 1)

In [45]:
aapl_data_scaled[0:60,0].shape

(60,)

In [46]:
y[0],X[1][len(X[1])-1]

(0.05358494912411915, 0.05358494912411915)

## Train test split
> This iis import because testing our model is as import as building a model so we use train and test split.
The test data will be usually 80% of the data i dataset and the test will be 20% of the data in the dataset

Sinces we are working with LSTM model we cant change the sequence of order so we don't use train test split module from sklearn

In [47]:
train_size = int(len(X)*0.8)
test_size = len(X) - train_size

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [48]:
X_train[:2],y_train[0]

([array([0.13375114, 0.1286109 , 0.13278292, 0.13030088, 0.13875052,
         0.15008715, 0.15131936, 0.16299047, 0.15545619, 0.15309733,
         0.15996267, 0.16610632, 0.16230396, 0.16429316, 0.16698649,
         0.16536695, 0.14889013, 0.16427554, 0.17598183, 0.17515447,
         0.14987592, 0.14837961, 0.16631755, 0.17089446, 0.17751333,
         0.16839474, 0.17107045, 0.16765538, 0.18101645, 0.17691481,
         0.17705567, 0.16658159, 0.17471438, 0.16887002, 0.15610751,
         0.12993116, 0.11215165, 0.12019645, 0.08652102, 0.08623935,
         0.13104019, 0.11433452, 0.13795834, 0.12067178, 0.11382401,
         0.07358251, 0.10732832, 0.08988328, 0.04200187, 0.09435458,
         0.0314046 , 0.05015228, 0.03925573, 0.03592868, 0.00857291,
         0.        , 0.03962541, 0.03723134, 0.05997502, 0.04113931]),
  array([0.1286109 , 0.13278292, 0.13030088, 0.13875052, 0.15008715,
         0.15131936, 0.16299047, 0.15545619, 0.15309733, 0.15996267,
         0.16610632, 0.16230396,

Here the X[i] data represent the last 60 values before the data y[i]

## Reshaping the Data
The data should be in the formate of 3D form
[batch_size, time_steps, units]
* Batch size determines how many samples are processed together in each training iteration.
* Time steps represent the number of steps or sequences in the input data.
* Features represent the different attributes or dimensions of the input data at each time step.

In [ ]:
np.array(X_train).shape

(756, 60)

In [ ]:
X_train[0].shape

(60,)

In [51]:
X_train, y_train = np.array(X_train),np.array(y_train)
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

In [52]:
X_train.shape

(756, 60, 1)

## Building a MODEL

#### Baseline Model

In [53]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply

model_1 = Sequential()

# Adding LSTM layers with return_sequences=True
model_1.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_1.add(LSTM(units=50, return_sequences=True))
# Adding self-attention mechanism
# The attention mechanism
attention = AdditiveAttention(name='attention_weight')
# Permute and reshape for compatibility
model_1.add(Permute((2, 1)))
model_1.add(Reshape((-1, X_train.shape[1])))
attention_result = attention([model_1.output, model_1.output])
multiply_layer = Multiply()([model_1.output, attention_result])
# Return to original shape
model_1.add(Permute((2, 1)))
model_1.add(Reshape((-1, 50)))

# Adding a Flatten layer before the final Dense layer
model_1.add(tf.keras.layers.Flatten())

# Final Dense layer
model_1.add(Dense(1))

In [54]:
model_1.compile(
    loss = tf.keras.losses.mean_squared_error,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['mean_squared_error','mean_absolute_error']
)

In [55]:
model_1.fit(X_train,
            y_train,
            epochs = 30,
            validation_split = 0.2,
            batch_size = 25)

Epoch 1/30
25/25 [==============================] - 6s 93ms/step - loss: 0.0446 - mean_squared_error: 0.0446 - mean_absolute_error: 0.1477 - val_loss: 0.0052 - val_mean_squared_error: 0.0052 - val_mean_absolute_error: 0.0609
Epoch 2/30
25/25 [==============================] - 1s 50ms/step - loss: 0.0072 - mean_squared_error: 0.0072 - mean_absolute_error: 0.0656 - val_loss: 0.0066 - val_mean_squared_error: 0.0066 - val_mean_absolute_error: 0.0671
Epoch 3/30
25/25 [==============================] - 2s 83ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - mean_absolute_error: 0.0572 - val_loss: 0.0051 - val_mean_squared_error: 0.0051 - val_mean_absolute_error: 0.0596
Epoch 4/30
25/25 [==============================] - 1s 59ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - mean_absolute_error: 0.0555 - val_loss: 0.0048 - val_mean_squared_error: 0.0048 - val_mean_absolute_error: 0.0575
Epoch 5/30
25/25 [==============================] - 1s 50ms/step - loss: 0.0049 - mean_squared_error

In [57]:
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
X_test.shape

(190, 60, 1)

In [58]:
test_loss = model_1.evaluate(X_test,y_test)
test_loss

6/6 [==============================] - 0s 17ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - mean_absolute_error: 0.0301


[0.001414355356246233, 0.001414355356246233, 0.03013734333217144]

## Performance Metrics

In [59]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

y_preds = model_1.predict(X_test)

mae = mean_absolute_error(y_test,y_preds)
mse = mean_squared_error(y_test,y_preds)

print("Mean absolute error:",mae),
print("Mean Square Error:",mse)

6/6 [==============================] - 1s 16ms/step
Mean absolute error: 0.03013735836830103
Mean Square Error: 0.0014143559350476807


## Improving the model

In [60]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply

model_2 = Sequential()
model_2.add(LSTM(units = 100,return_sequences = True,input_shape = (X_train.shape[1],1)))
model_2.add(Dropout(0.2))
model_2.add(LSTM(units = 100,return_sequences = True))
model_2.add(Dropout(0.2))
model_2.add(LSTM(units = 100,return_sequences = True,input_shape = (X_train.shape[1],1)))
model_2.add(Dropout(0.2))
model_2.add(LSTM(units = 100,return_sequences = True))
model_2.add(Dropout(0.2))
model_2.add(tf.keras.layers.Flatten())
model_2.add(Dense(1))

In [61]:
model_2.compile(
   loss = tf.keras.losses.mean_squared_error,
   optimizer = tf.keras.optimizers.Adam(),
   metrics = ['mean_squared_error','mean_absolute_error']
)

In [62]:
history_2 = model_2.fit(
    X_train,
    y_train,
    epochs = 30,
    batch_size = 25,
    validation_split = 0.2,
)

Epoch 1/30
25/25 [==============================] - 15s 263ms/step - loss: 0.0561 - mean_squared_error: 0.0561 - mean_absolute_error: 0.1830 - val_loss: 0.0196 - val_mean_squared_error: 0.0196 - val_mean_absolute_error: 0.1189
Epoch 2/30
25/25 [==============================] - 4s 162ms/step - loss: 0.0096 - mean_squared_error: 0.0096 - mean_absolute_error: 0.0779 - val_loss: 0.0069 - val_mean_squared_error: 0.0069 - val_mean_absolute_error: 0.0700
Epoch 3/30
25/25 [==============================] - 5s 207ms/step - loss: 0.0071 - mean_squared_error: 0.0071 - mean_absolute_error: 0.0663 - val_loss: 0.0061 - val_mean_squared_error: 0.0061 - val_mean_absolute_error: 0.0662
Epoch 4/30
25/25 [==============================] - 4s 163ms/step - loss: 0.0072 - mean_squared_error: 0.0072 - mean_absolute_error: 0.0663 - val_loss: 0.0059 - val_mean_squared_error: 0.0059 - val_mean_absolute_error: 0.0639
Epoch 5/30
25/25 [==============================] - 5s 203ms/step - loss: 0.0071 - mean_squared

In [64]:
model_2.evaluate(X_test,y_test)

6/6 [==============================] - 0s 58ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - mean_absolute_error: 0.0311


[0.001483336091041565, 0.001483336091041565, 0.03113434463739395]

## Making Predictions

model_2:
loss: 0.0015 - mean_squared_error: 0.0015 - mean_absolute_error: 0.0311
[0.001483336091041565, 0.001483336091041565, 0.03113434463739395]

model_1:
6/6 [==============================] - 0s 17ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - mean_absolute_error: 0.0301
[0.001414355356246233, 0.001414355356246233, 0.03013734333217144]

In [67]:
test_data = yf.download('AAPL',start = '2020-01-01',end = '2024-04-27')
test_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.059433,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.349152,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.925636,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.582680,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.750252,132079200
...,...,...,...,...,...,...
2024-04-22,165.520004,167.259995,164.770004,165.839996,165.839996,48116400
2024-04-23,165.350006,167.050003,164.919998,166.899994,166.899994,49537800
2024-04-24,166.539993,169.300003,166.210007,169.020004,169.020004,48251800


In [68]:
index = test_data.index.get_loc('2024-04-26')
index

1086

In [69]:
len(test_data)

1087

In [70]:
X_model_prediction = test_data['Close'][index-60:index].values
len(X_model_prediction)

60

In [71]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_model_prediction = scaler.fit_transform(X_model_prediction.reshape(-1,1))

In [72]:
X_model_prediction.shape

(60, 1)

In [73]:
X_model_prediction = np.array(X_model_prediction)
X_model_prediction = np.reshape(X_model_prediction,(1,60,1))
X_model_prediction.shape

(1, 60, 1)

In [74]:
model_2.predict(X_model_prediction)

1/1 [==============================] - 2s 2s/step


array([[0.30574664]], dtype=float32)

In [75]:
y_model_prediction_true = np.array(test_data.iloc[index]['Close'])

In [77]:
y_model_prediction_true

array(169.30000305)

In [78]:
normalized_prediction = np.array([[0.30574664]])
actual_prediction = scaler.inverse_transform(normalized_prediction)

print("Actual regression value predicted by the model_2:", actual_prediction)
print("Actual value:",y_model_prediction_true)

Actual regression value predicted by the model_2: [[172.4632766]]
Actual value: 169.3000030517578


In [79]:
model_1.predict(X_model_prediction)

1/1 [==============================] - 0s 26ms/step


array([[0.22062917]], dtype=float32)

In [80]:
y_model_prediction_true

array(169.30000305)

In [83]:
normalized_prediction = np.array([[0.22062917]])
actual_prediction = scaler.inverse_transform(normalized_prediction)

print("Actual regression value predicted by the model_1:", actual_prediction)
print("Actual value:",y_model_prediction_true)

Actual regression value predicted by the model_1: [[170.38555885]]
Actual value: 169.3000030517578
